In [4]:
import sys
import glob
from pyspark import SparkContext, SparkConf


In [2]:

conf = SparkConf()
sc = SparkContext.getOrCreate(conf = conf)
stop_words = {"they", "he", "she", "it", "the", "as", "is", "and"}
lines_rdd = sc.wholeTextFiles("data/*")
words_rdd = lines_rdd.flatMap(lambda x: [(x[0], word) for word in x[1].split()])
words_rdd = words_rdd.filter(lambda word:word[1] not in stop_words)
key_value_rdd = words_rdd.map(lambda x: ((x[0],x[1]), 1))
word_counts_rdd = key_value_rdd.reduceByKey(lambda x,y: x + y)
flipped_rdd = word_counts_rdd.map(lambda x: (x[0][1], [(x[0][0], x[1])]))

21/11/27 08:00:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
res_rdd = flipped_rdd.reduceByKey(lambda x, y: x + y)
res_rdd.coalesce(1).saveAsTextFile("output")